In [1]:
import glob
import warnings
import tarfile
import cv2

import keras
import keras.backend as K

from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, LeakyReLU
from keras.layers import BatchNormalization, Activation, Conv2D
from keras.layers import GlobalAveragePooling2D, Lambda
from keras.optimizers import Adam, RMSprop

from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
from keras.preprocessing.image import ImageDataGenerator 
from keras.utils import to_categorical, Sequence
from keras.callbacks import ModelCheckpoint

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score
from keras.models import load_model, model_from_json

import matplotlib.pyplot as plt

import os
import numpy as np
import pandas as pd
from PIL import Image
from cv2 import resize

import tensorflow as tf
from keras.applications import ResNet50
from keras import regularizers
import requests
import threading
import random
import time
import urllib
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.utils import multi_gpu_model

from collections import Counter

%matplotlib inline

print('Keras version:', keras.__version__)
# print(os.listdir('SageMaker'))

warnings.simplefilter('default')
!pwd
import argparse
import glob
import pickle
import sys
import warnings

import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from scipy.spatial import cKDTree
from skimage.feature import plot_matches
from skimage.measure import ransac
from skimage.transform import AffineTransform
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.python.platform import app

warnings.simplefilter('default')

Using TensorFlow backend.


Keras version: 2.2.4
/home/ec2-user/SageMaker


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  return f(*args, **kwds)
W0517 18:40:03.740313 139693473142592 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [2]:
train_path = './data/landmarks/train/train/'
test_path = './data/landmarks/test/test/'
train_images = glob.glob(train_path+'*.jpg')
test_images = glob.glob(test_path+'*.jpg')
print(len(train_images))
print(len(test_images))
sample_submission = pd.read_csv('./data/landmarks/recognition_sample_submission.csv')
sample_submission.shape

4130318
112821


(117703, 2)

In [3]:
train_image_ids = [image_file.replace(
    '.jpg', '').replace(train_path, '') for image_file in train_images]

train_df = pd.DataFrame(index=list(range(0,len(train_image_ids))))
train_df['filename'] = pd.Series(train_images, index=list(range(0,len(train_image_ids))))
train_df['ids'] = train_image_ids
test_image_ids = [image_file.replace(
    '.jpg', '').replace(test_path, '') for image_file in test_images]
test_df = pd.DataFrame(index=list(range(0,len(test_image_ids))))
test_df['filename'] = pd.Series(test_images, index=list(range(0,len(test_image_ids))))
test_df['ids'] = test_image_ids

In [4]:
train = pd.read_csv("./data/train.csv",index_col='id')
print(train.head())
print(train.shape)
print("Number of classes {}".format(len(train.landmark_id.unique())))

NUM_THRESHOLD = 42

counts = dict(Counter(train['landmark_id']))
landmarks_dict = {x:[] for x in train.landmark_id.unique() if counts[x] >= NUM_THRESHOLD}
NUM_CLASSES = len(landmarks_dict)
print("Total number of valid classes: {}".format(NUM_CLASSES))

i = 0
landmark_to_idx = {}
idx_to_landmark = []
for k in landmarks_dict:
    landmark_to_idx[k] = i
    idx_to_landmark.append(k)
    i += 1
    
train['filename'] = pd.Series(train_images, index=train_image_ids)

train = train.dropna(axis=0)
print(train.isna().sum())

all_urls = train['url'].tolist()
all_filenames= train['filename'].tolist()
all_landmarks = train['landmark_id'].tolist()
valid_urls_dict = {x[0].split("/")[-1]:landmark_to_idx[x[1]] for x in zip(all_urls, all_landmarks) if x[1] in landmarks_dict}
valid_filenames_dict = {x[0].split('/')[-1]:landmark_to_idx[x[1]] for x in zip(all_filenames, all_landmarks) if x[1] in landmarks_dict}
valid_urls_list = [x[0] for x in zip(all_urls, all_landmarks) if x[1] in landmarks_dict]
valid_filenames_list = [x[0] for x in zip(all_filenames, all_landmarks) if x[1] in landmarks_dict]

NUM_EXAMPLES = len(valid_urls_list)
print("Total number of valid examples: {}".format(NUM_EXAMPLES))

                                                                url  \
id                                                                    
6e158a47eb2ca3f6  https://upload.wikimedia.org/wikipedia/commons...   
202cd79556f30760  http://upload.wikimedia.org/wikipedia/commons/...   
3ad87684c99c06e1  http://upload.wikimedia.org/wikipedia/commons/...   
e7f70e9c61e66af3  https://upload.wikimedia.org/wikipedia/commons...   
4072182eddd0100e  https://upload.wikimedia.org/wikipedia/commons...   

                  landmark_id  
id                             
6e158a47eb2ca3f6       142820  
202cd79556f30760       104169  
3ad87684c99c06e1        37914  
e7f70e9c61e66af3       102140  
4072182eddd0100e         2474  
(4132914, 2)
Number of classes 203094
Total number of valid classes: 23215
url            0
landmark_id    0
filename       0
dtype: int64
Total number of valid examples: 2296997


In [5]:
test_info_full = pd.read_csv('./data/test.csv', index_col='id')
test_info_full.head()

test_info = test_info_full.loc[test_image_ids]
test_info['filename'] = pd.Series(test_images, index=test_image_ids)

test_info.head()

,url,filename
id,,
fa8d5a81a16f2f2f,https://lh3.googleusercontent.com/-_SAJTBt3Y64...,./data/landmarks/test/test/fa8d5a81a16f2f2f.jpg
b81a0a45f9b1ee97,https://lh3.googleusercontent.com/-9sFSIOCzIOs...,./data/landmarks/test/test/b81a0a45f9b1ee97.jpg
570e28cc63fab858,https://lh3.googleusercontent.com/-7Eld7yUfAB0...,./data/landmarks/test/test/570e28cc63fab858.jpg
b8bc63608b5fef1a,https://lh3.googleusercontent.com/-JdgzGjeS9NE...,./data/landmarks/test/test/b8bc63608b5fef1a.jpg
54cfd1f5f683b966,https://lh3.googleusercontent.com/-krCM7YZ3FpU...,./data/landmarks/test/test/54cfd1f5f683b966.jpg


In [6]:
n_cat = 203094 #number of unique classes (yikes)
n_valid_classes = 23215
input_shape = (99,99)
batch_size = 48
batch_size_predict = 96

In [7]:
label_encoder = LabelEncoder()
one_hot_encoder = OneHotEncoder(sparse=True, n_values=n_cat)

train['label'] = label_encoder.fit_transform(train['landmark_id'].values)
train['one_hot'] = one_hot_encoder.fit_transform(
                    train['label'].values.reshape(-1, 1))

In [8]:
def load_images(info, input_shape = input_shape):
    input_shape = tuple(input_shape)
    imgs = np.zeros((len(info), input_shape[0], input_shape[1], 3))

    for i in range(len(info)):
        fname = info.iloc[i]['filename']
        try:
            img = cv2.cvtColor(
                  cv2.resize(cv2.imread(fname),input_shape),
                  cv2.COLOR_BGR2RGB)
        except:
            warnings.warn('Warning: could not read image: '+ fname +
                          '. Use black img instead.')
            img = np.zeros((input_shape[0], input_shape[1], 3))
        imgs[i,:,:,:] = img
    
    return imgs
def load_cropped_images(info, crop_p=0.2, crop='random'):
    new_res = np.array([int(input_shape[0]*(1+crop_p)), int(input_shape[1]*(1+crop_p))])
    if crop == 'random':
        cx0 = np.random.randint(new_res[0] - input_shape[0], size=len(info))
        cy0 = np.random.randint(new_res[1] - input_shape[1], size=len(info))
    else:
        if crop == 'central':
            cx0, cy0 = (new_res - input_shape) // 2                
        if crop == 'upper left':
            cx0, cy0 = 0, 0
        if crop == 'upper right':
            cx0, cy0 = new_res[1] - input_shape[1], 0
        if crop == 'lower left':
            cx0, cy0 = 0, new_res[0] - input_shape[0]
        if crop=='lower right':
            cx0, cy0 = new_res - input_shape        
        cx0 = np.repeat(np.expand_dims(cx0, 0), len(info))
        cy0 = np.repeat(np.expand_dims(cy0, 0), len(info))

    cx1 = cx0 + input_shape[0]
    cy1 = cy0 + input_shape[1]
    
    raw_imgs = load_images(info, input_shape=tuple(new_res))
    
    cropped_imgs = np.zeros((len(info), input_shape[0], input_shape[1], 3))
    for ind in range(len(info)):
        cropped_imgs[ind,:,:,:] = raw_imgs[ind,
                                           cy0[ind]:cy1[ind],
                                           cx0[ind]:cx1[ind], :]
    
    return cropped_imgs


def get_image_gen(info_arg, 
                  shuffle=True, 
                  image_aug=True, 
                  eq_dist=False, 
                  n_ref_imgs=16, 
                  crop_prob=0.5, 
                  crop_p=0.5):
    if image_aug:
        datagen = ImageDataGenerator(
            rotation_range=4.,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.5,
            channel_shift_range=25,
            horizontal_flip=True,
            fill_mode='nearest')
        
        if crop_prob > 0:
            datagen_crop = ImageDataGenerator(
                rotation_range=4.,
                shear_range=0.2,
                zoom_range=0.1,
                channel_shift_range=20,
                horizontal_flip=True,
                fill_mode='nearest')
        
    count = len(info_arg)
    while True:
        if eq_dist:
            def sample(df):
                return df.sample(min(n_ref_imgs, len(df)))
            info = info_arg.groupby('landmark_id', group_keys=False).apply(sample)
        else:
            info = info_arg
        print('Generate', len(info), 'for the next round.')
        
        #shuffle data
        if shuffle and count >= len(info):
            info = info.sample(frac=1)
            count = 0
            
        # load images
        for ind in range(0,len(info), batch_size):
            count += batch_size

            y = info['landmark_id'].values[ind:(ind+batch_size)]
            
            if np.random.rand() < crop_prob:
                imgs = load_cropped_images(info.iloc[ind:(ind+batch_size)], 
                                           crop_p=crop_p*np.random.rand() + 0.01, 
                                           crop='random')
                if image_aug:
                    cflow = datagen_crop.flow(imgs, 
                                              y, 
                                              batch_size=imgs.shape[0], 
                                              shuffle=False)
                    imgs, y = next(cflow)                    
            else:
                imgs = load_images(info.iloc[ind:(ind+batch_size)])
                if image_aug:
                    cflow = datagen.flow(imgs, 
                                       y, 
                                       batch_size=imgs.shape[0], 
                                       shuffle=False)
                    imgs, y = next(cflow)             

            imgs = preprocess_input(imgs)
    
            y_l = label_encoder.transform(y[y>=0.])        
            y_oh = np.zeros((len(y), n_cat))
            y_oh[y >= 0., :] = one_hot_encoder.transform(y_l.reshape(-1,1)).todense()
                    
            yield imgs, y_oh
            
train_gen = get_image_gen(train, 
                          eq_dist=True, 
                          n_ref_imgs=512, 
                          crop_prob=0.3, 
                          crop_p=0.5)

In [49]:
_DISTANCE_THRESHOLD = 0.8

def load_delf_features(img_id, ddir):
    locations = np.load(ddir + img_id + '_loc.npy')
    descriptions = np.load(ddir + img_id + '_desc.npy')
    return locations, descriptions

In [10]:
def compare_imgs_1_preloaded(locations_1, 
                             descriptors_1, 
                             d1_tree, 
                             img_id_2, 
                             dir_2='delf-train/'):
    # Read features.
    num_features_1 = locations_1.shape[0]
    locations_2, descriptors_2 = load_delf_features(img_id_2, dir_2)
    num_features_2 = locations_2.shape[0]

    if len(locations_1)*len(locations_2)==0:
        return 0
    
    _, indices = d1_tree.query(
            descriptors_2, 
            distance_upper_bound=_DISTANCE_THRESHOLD)

    if len(indices)==0:
        return 0
    
    # Select feature locations for putative matches.       
    locations_2_to_use = np.array([
      locations_2[i,]
      for i in range(num_features_2)
      if indices[i] != num_features_1
    ])
    locations_1_to_use = np.array([
      locations_1[indices[i],]
      for i in range(num_features_2)
      if indices[i] != num_features_1
    ])

    # Perform geometric verification using RANSAC.                                                   
    
    if len(locations_1_to_use)*len(locations_2_to_use)==0:
        return 0

    _, inliers = ransac(
      (locations_1_to_use, locations_2_to_use),
      AffineTransform,
      min_samples=3,
      residual_threshold=20,
      max_trials=1000)

    if inliers is None:
        score = 0.
    else:
        score = sum(inliers)

    return score

In [25]:
def compare_imgs(img_id_1, 
                 img_id_2, 
                 dir_1, dir_2='delf-train/', 
                 plot=False, 
                 img_dir_1=None, 
                 img_dir_2=None):
    # Read features.
    _DISTANCE_THRESHOLD = 0.8
    locations_1, descriptors_1 = load_delf_features(img_id_1, dir_1)
    num_features_1 = locations_1.shape[0]
    
    locations_2, descriptors_2 = load_delf_features(img_id_2, dir_2)
    num_features_2 = locations_2.shape[0]
    
    if len(locations_1)*len(locations_2)==0:
        return 0
    
    d1_tree = cKDTree(descriptors_1)
    _, indices = d1_tree.query(
      descriptors_2, distance_upper_bound=_DISTANCE_THRESHOLD)

    # Select feature locations for putative matches.       
    locations_2_to_use = np.array([
      locations_2[i,]
      for i in range(num_features_2)
      if indices[i] != num_features_1
    ])
    locations_1_to_use = np.array([
      locations_1[indices[i],]
      for i in range(num_features_2)
      if indices[i] != num_features_1
    ])

    # Perform geometric verification using RANSAC.                                                   
#     try:  
    _, inliers = ransac(
          (locations_1_to_use, locations_2_to_use),
          AffineTransform,
          min_samples=3,
          residual_threshold=20,
          max_trials=1000)

    if len(locations_1_to_use)*len(locations_2_to_use)==0:
        score = 0
    elif inliers is None:
        score = 0.   
    else:
        score = sum(inliers)       
    
#     if plot:
#         _, ax = plt.subplots()
#         img_1=cv2.cvtColor(
#                     cv2.resize(cv2.imread(img_dir_1+img_id_1+'.jpg'),input_shape),
#                     cv2.COLOR_BGR2RGB)/255.
#         img_2=cv2.cvtColor(
#                     cv2.resize(cv2.imread(img_dir_2+img_id_2+'.jpg'),input_shape),
#                     cv2.COLOR_BGR2RGB)/255. 
        
#         inlier_idxs = np.nonzero(inliers)[0]
#         plot_matches(
#           ax,
#           img_1,
#           img_2,
#           locations_1_to_use,
#           locations_2_to_use,
#           np.column_stack((inlier_idxs, inlier_idxs)),
#           matches_color='b')
#         ax.axis('off')
#         ax.set_title('DELF correspondences')

    return score

In [23]:
def compare_features(locations_1, descriptors_1, locations_2, descriptors_2, d1_tree):

    num_features_1 = locations_1.shape[0]
    num_features_2 = locations_2.shape[0]
    
    if num_features_1 * num_features_2 == 0:
        return 0
    
    _, indices = d1_tree.query(
      descriptors_2, distance_upper_bound=_DISTANCE_THRESHOLD)

    if len(indices)==0:
        return 0
    
    # Select feature locations for putative matches.           
    locations_2_to_use = np.array([
      locations_2[i,]
      for i in range(num_features_2)
      if indices[i] != num_features_1
    ])
    locations_1_to_use = np.array([
      locations_1[indices[i],]
      for i in range(num_features_2)
      if indices[i] != num_features_1
    ])


    # Perform geometric verification using RANSAC.                                                   
    if len(locations_1_to_use) * len(locations_2_to_use) == 0:
        return 0
    
    _, inliers = ransac(
      (locations_1_to_use, locations_2_to_use),
      AffineTransform,
      min_samples=3,
      residual_threshold=20,
      max_trials=1000)

    if inliers is None:
        score = 0.
    else:
        score = sum(inliers)
    return score

In [34]:
# score=compare_imgs(train[train['landmark_id'] == 4831].sample(1).index[0],
#                    train[train['landmark_id'] == 4302].sample(1).index[0], 
#                    dir_1='delf-train/', 
#                    dir_2='delf-train/', 
#                    img_dir_1='./data/landmarks/train/train/', 
#                    img_dir_2='./data/landmarks/train/train/')
# print(score)

In [46]:
r_train_path = './delf-train/'
r_train_image_files = glob.glob(r_train_path + '*_loc.npy')
r_train_image_ids = [image_file.replace(
    '_loc.npy', '').replace(r_train_path, '') for image_file in r_train_image_files]
r_train_info=train.loc[r_train_image_ids]
r_train_info['filename'] = pd.Series(dev_image_files, index=dev_image_ids)

In [47]:
r_train_info.head()

,url,landmark_id,filename,label,one_hot
id,,,,,
9e10f9e254ae67c8,https://upload.wikimedia.org/wikipedia/commons...,10562,NaN,10562,"(0, 142820)\t1.0\n (1, 104169)\t1.0\n (2, ..."
fa2eaf85a62f7271,https://upload.wikimedia.org/wikipedia/commons...,8262,NaN,8262,"(0, 142820)\t1.0\n (1, 104169)\t1.0\n (2, ..."
8933414bc50c4d25,http://upload.wikimedia.org/wikipedia/commons/...,3591,NaN,3591,"(0, 142820)\t1.0\n (1, 104169)\t1.0\n (2, ..."
81d9b65c9b38d5c1,https://upload.wikimedia.org/wikipedia/commons...,16912,NaN,16912,"(0, 142820)\t1.0\n (1, 104169)\t1.0\n (2, ..."
48e2cd96847945e7,https://upload.wikimedia.org/wikipedia/commons...,7201,NaN,7201,"(0, 142820)\t1.0\n (1, 104169)\t1.0\n (2, ..."


In [48]:
x1,y1 = load_delf_features(r_train_info.index[1], 'delf-train/')
x2,y2 = load_delf_features(r_train_info.index[2], 'delf-train/')